### Importing libraries

In [ ]:
from barmnist import BarMNIST
from torchvision import transforms
import torch
from cdm.utils import visualize_image
import os
from diffusers import UNet2DModel, DDPMScheduler
import logging
from dotenv import load_dotenv
from cdm.scm import EdgeType, Vertex, Edge, SCM

# Setup the logger
load_dotenv()
logger = logging.getLogger(__name__)
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler("celeba.log"),  # Log to file
        logging.StreamHandler()          # Log to console
    ],
    datefmt="%Y-%m-%d %H:%M:%S"
)
logger.setLevel(os.getenv('LOG_LEVEL'))
logger.info(os.getenv('LOG_LEVEL'))

/home/asardesai/Projects/Causal-Diffusion-Model/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-18 15:53:36 - INFO - DEBUG


In [7]:
BATCH_SIZE = int(os.getenv('BATCH_SIZE'))
NUM_EPOCHS = int(os.getenv('NUM_EPOCHS'))
MODEL_PATH = 'mnist_model.pt'
DEVICE_NAME = os.getenv('DEVICE')
DEVICE = torch.device(DEVICE_NAME) if torch.cuda.is_available() else 'cpu'
IMAGE_RESOLUTION = 32 # Dimensions of image to be passed into VAE
IMAGE_CHANNELS = 3 # RGB
LATENT_SPACE_RESOLUTION = int(IMAGE_RESOLUTION / 8) # Size of the latent space
LATENT_SPACE_CHANNELS = 4 # Channel dimension of latent space
LEARNING_RATE = 1e-4

### Load the Dataset

In [4]:
cg = 'frontdoor'

BarMNIST(cg=cg, ow=True)
trans_f = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                                ])
train_set = BarMNIST(cg=cg, root='./', env='train', transform=trans_f)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True,
                                            drop_last=True)

Preparing BAR MNIST
Causal Diagram:frontdoor
BAR MNISTfrontdoor dataset already exists


In [5]:
x_sample = None
for idx, (x, y) in enumerate(train_loader):
    x_sample = x[0]
    print(y[0])
    break

visualize_image(x_sample)

tensor([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 0.])


In [8]:
# Create the network or load an existing one
if os.path.exists(MODEL_PATH):
    net = torch.load(MODEL_PATH, weights_only=False, map_location=DEVICE)
    logger.info("Loading model from checkpoint...")
else:
    net = UNet2DModel(
        sample_size=IMAGE_RESOLUTION,  # the target image resolution
        in_channels=IMAGE_CHANNELS,  # the number of input channels, 3 for RGB images
        out_channels=IMAGE_CHANNELS,  # the number of output channels
        layers_per_block=2,  # how many ResNet layers to use per UNet block
        block_out_channels=(128, 256, 512, 512),  # Roughly matching our basic unet example
        down_block_types=(
            "AttnDownBlock2D",  # a regular ResNet downsampling block
            "AttnDownBlock2D",  # a ResNet downsampling block with spatial self-attention
            "AttnDownBlock2D",
            "DownBlock2D"
        ),
        up_block_types=(
            "UpBlock2D",
            "AttnUpBlock2D",  # a ResNet upsampling block with spatial self-attention
            "AttnUpBlock2D",  # a regular ResNet upsampling block
            "AttnUpBlock2D",  # a regular ResNet upsampling block
        ),
    )  # <<<
    logger.info("Initializing a new model...")
net.to(DEVICE)

2025-04-18 15:56:45 - INFO - Initializing a new model...


UNet2DModel(
  (conv_in): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (time_proj): Timesteps()
  (time_embedding): TimestepEmbedding(
    (linear_1): Linear(in_features=128, out_features=512, bias=True)
    (act): SiLU()
    (linear_2): Linear(in_features=512, out_features=512, bias=True)
  )
  (down_blocks): ModuleList(
    (0): AttnDownBlock2D(
      (attentions): ModuleList(
        (0-1): 2 x Attention(
          (group_norm): GroupNorm(32, 128, eps=1e-05, affine=True)
          (to_q): Linear(in_features=128, out_features=128, bias=True)
          (to_k): Linear(in_features=128, out_features=128, bias=True)
          (to_v): Linear(in_features=128, out_features=128, bias=True)
          (to_out): ModuleList(
            (0): Linear(in_features=128, out_features=128, bias=True)
            (1): Dropout(p=0.0, inplace=False)
          )
        )
      )
      (resnets): ModuleList(
        (0-1): 2 x ResnetBlock2D(
          (norm1): GroupNorm(32, 128, eps=1

### Initialize the SCM Model

In [ ]:
d_vertex = Vertex('Digit', (BATCH_SIZE, 10))
c_vertex = Vertex('Color', (BATCH_SIZE, 2))
b_vertex = Vertex('Bar', (BATCH_SIZE, 2))
edge_list = [
    Edge(d_vertex, c_vertex, EdgeType.DIRECTED),
    Edge(c_vertex, b_vertex, EdgeType.DIRECTED),
]


### 